In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 4.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

from braindecode.torch_ext.util import np_to_var

In [ ]:
import torch as th

In [ ]:
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')

In [ ]:
from autodiag.dataset import load_data, DiagnosisSet, get_all_sorted_file_names_and_labels

In [ ]:
all_file_names, labels = get_all_sorted_file_names_and_labels()

In [ ]:
from numpy.random import RandomState
rng = RandomState(384734)
in_var = np_to_var(rng.randn(3,5,18,2), dtype=np.float32)
percentile = th.kthvalue(th.abs(in_var.view(-1)), int(0.95 * in_var.numel()))
cutoff = percentile * 2
# store das die ganze zeit, und beim forward pass clampe die werte dann zu +-cutoff

In [ ]:
lengths = np.load(
            '/home/schirrmr/code/auto-diagnosis/sorted-recording-lengths.npy')
mask = lengths < 35 * 60
cleaned_file_names = np.array(all_file_names)[mask]
cleaned_labels = labels[mask]

In [ ]:
sec_to_cut = 60
preproc_functions = []
duration_recording_mins = 3
preproc_functions.append(
    lambda data, fs: (data[:, int(sec_to_cut * fs):-int(
        sec_to_cut * fs)], fs))
preproc_functions.append(
    lambda data, fs: (data[:, :int(duration_recording_mins * 60 * fs)], fs))

In [ ]:
import torch.nn.functional as F
def compute_min_max_diffs(x, window_len, cuda):
    x_var = np_to_var(x)
    if x_var.dim() == 2:
        x_var = x_var.unsqueeze(0)
    if cuda:
        x_var = x_var.cuda()

    maxs = F.max_pool1d(x_var,window_len, stride=1)
    mins = F.max_pool1d(-x_var,window_len, stride=1)

    diffs = maxs + mins
    return diffs

In [ ]:
train_inds = np.int32(np.arange(10))
valid_inds = np.int32(np.arange(10,12))
test_inds = np.int32(np.arange(12,15))

In [ ]:
import resampy
sec_to_cut = 60
preproc_functions = []
duration_recording_mins = 3
sampling_freq = 100
preproc_functions.append(
    lambda data, fs: (data[:, int(sec_to_cut * fs):-int(
        sec_to_cut * fs)], fs))
preproc_functions.append(
    lambda data, fs: (data[:, :int(duration_recording_mins * 60 * fs)], fs))
preproc_functions.append(lambda data, fs: (resampy.resample(data, fs,
                                                            sampling_freq,
                                                            axis=1,
                                                            filter='kaiser_fast'),
                                           sampling_freq))

In [ ]:
from copy import deepcopy
import resampy
def preproc_set_further(the_set, start_fs, preproc_functions):
    the_set = deepcopy(the_set)
    new_X = []
    for x in the_set.X:
        fs = start_fs
        for preproc_fn in preproc_functions:
            x, fs = preproc_fn(x, fs)
        new_X.append(x)
    the_set.X = new_X
    return the_set
        

In [ ]:
from braindecode.datautil.signal_target import SignalAndTarget
def create_set(inds):
    X = []
    for i in inds:
        log.info("Load {:s}".format(cleaned_file_names[i]))
        x = load_data(cleaned_file_names[i], preproc_functions)
        X.append(x)
    y = cleaned_labels[inds].astype(np.int64)
    return SignalAndTarget(X, y)

In [ ]:

from autodiag.iterator import ModifiedIterator
from autodiag.batch_modifier import RemoveMinMaxDiff

        
from autodiag.clean import set_jumps_to_zero

In [ ]:
train_set = create_set(train_inds)
valid_set = create_set(valid_inds)
test_set = create_set(test_inds)

In [ ]:
new_preproc_fn = []#lambda data, fs: (remove_jumps(data,200,400,True,True), fs)]
start_fs = 100
#new_preproc_fn.append(lambda data, fs : (np.clip(data,-200,200),fs))
new_train_set = preproc_set_further(train_set, start_fs, new_preproc_fn)
new_valid_set = preproc_set_further(valid_set, start_fs, new_preproc_fn)
new_test_set = preproc_set_further(test_set, start_fs, new_preproc_fn)

In [ ]:
from braindecode.torch_ext.modules import Expression
from torch import nn
def net_with_more_layers(net, n_blocks_to_add,
                        later_pool_class):
    def add_conv_pool_block(model, n_filters_before,
                            n_filters, filter_length, block_nr):
        suffix = '_{:d}'.format(block_nr)
        model.add_module('drop' + suffix,
                         nn.Dropout(p=net.drop_prob))
        model.add_module('conv' + suffix.format(block_nr),
                         nn.Conv2d(n_filters_before, n_filters,
                                   (filter_length, 1),
                                   stride=1, bias=not net.batch_norm))
        if net.batch_norm:
            model.add_module('bnorm' + suffix,
                         nn.BatchNorm2d(n_filters,
                                        momentum=net.batch_norm_alpha,
                                        affine=True,
                                        eps=1e-5))
        model.add_module('nonlin' + suffix,
                         Expression(net.later_nonlin))

        model.add_module('pool' + suffix,
                         later_pool_class(
                             kernel_size=(net.pool_time_length, 1),
                             stride=(net.pool_time_stride, 1)))
        model.add_module('pool_nonlin' + suffix,
                         Expression(net.later_pool_nonlin))

    model = net.create_network()
    from torch import nn
    larger_model = nn.Sequential()
    for name, module in model.named_children():
            if name == 'conv_classifier':
                break
            larger_model.add_module(name, module)
    for i_block in range(n_blocks_to_add):
        add_conv_pool_block(larger_model, net.n_filters_4, net.n_filters_4,
                            10,i_block+5)

    after_classifier = False
    for name, module in model.named_children():
            if name == 'conv_classifier':
                after_classifier = True
            if after_classifier:
                larger_model.add_module(name, module)
    return larger_model

In [ ]:
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.deep4 import Deep4Net
cuda = True
input_time_length = 1200
final_conv_length = 1
set_random_seeds(seed=20170629, cuda=cuda)
# This will determine how many crops are processed in parallel
n_classes = 2
in_chans = 21
pool_stride = 3
net =  Deep4Net(in_chans=in_chans, n_classes=n_classes,
                input_time_length=input_time_length,
                final_conv_length=final_conv_length,
                pool_time_length=pool_stride,
                pool_time_stride=pool_stride,
                n_filters_2=50, n_filters_3=80,
                n_filters_4=120,
                )
model = net_with_more_layers(net, 3, nn.MaxPool2d)

In [ ]:
from torch import optim
from braindecode.models.util import to_dense_prediction_model
input_time_length = 18000
optimizer = optim.Adam(model.parameters())#(model.parameters(), grad_clip_std_factor=2)#optim.Adam(model.parameters())
to_dense_prediction_model(model)
log.info("Model:\n{:s}".format(str(model)))
if cuda:
    model.cuda()
# determine output size
test_input = np_to_var(
    np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]

In [ ]:
import torch.nn.functional as F
from braindecode.experiments.experiment import Experiment
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.experiments.monitors import (RuntimeMonitor, LossMonitor,
                                              CroppedTrialMisclassMonitor,
                                              MisclassMonitor)
from braindecode.experiments.stopcriteria import MaxEpochs

batch_size = 32
log.info("{:d} predictions per input/trial".format(n_preds_per_input))
iterator = CropsFromTrialsIterator(batch_size=batch_size,
                                   input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)
loss_function = lambda preds, targets: F.nll_loss(
            th.mean(preds, dim=2)[:,:,0], targets)
model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length),
            RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
batch_modifier = RemoveMinMaxDiff(300,clip_max_abs=True, set_zero=True)#None
iterator = ModifiedIterator(iterator, batch_modifier)
batch_modifier=None
exp = Experiment(model, new_train_set, new_valid_set, new_test_set, iterator,
                 loss_function, optimizer, model_constraint,
                 monitors, stop_criterion,
                 remember_best_column='valid_misclass',
                 run_after_early_stop=False, batch_modifier=batch_modifier, cuda=cuda)

In [ ]:
exp.setup_training()
exp.run_until_early_stop()

In [ ]:
np.std(np.array(train_set.X))

In [ ]:
np.std(np.array(new_train_set.X))

In [ ]:
plt.plot(exp.epochs_df.train_loss)

In [ ]:
plt.plot(exp.epochs_df.train_loss)

In [ ]:
plt.plot(exp.epochs_df.train_loss)

In [ ]:
plt.plot(exp.epochs_df.train_loss)

In [ ]:
flat_test_X = np.array(test_set.X).reshape(-1)

In [ ]:
seaborn.distplot(flat_test_X)

In [ ]:
np.histogram(flat_test_X)

In [ ]:
np.histogram(flat_test_X, density=True)